In [18]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage,HumanMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [4]:
# llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llm=ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
prompt=ChatPromptTemplate.from_template(
    template="Tell me a Short joke about {topic}"
)

In [6]:
outParser=StrOutputParser()

In [7]:
chain=prompt|llm|StrOutputParser()

In [8]:
question={"topic":"Virat Kohli"}

In [9]:
chain.invoke(input=question)

'Why did Virat Kohli bring a ladder to the cricket match? \n\nBecause he heard the bowlers were pitching tents on the field!'

<h3>Prompt</h3>

In [10]:
promptValue=prompt.invoke(input=question)
promptValue

ChatPromptValue(messages=[HumanMessage(content='Tell me a Short joke about Virat Kohli', additional_kwargs={}, response_metadata={})])

In [11]:
promptValue.to_messages()

[HumanMessage(content='Tell me a Short joke about Virat Kohli', additional_kwargs={}, response_metadata={})]

In [12]:
promptValue.to_string()

'Human: Tell me a Short joke about Virat Kohli'

In [14]:
promptValue.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'prompts', 'chat', 'ChatPromptValue'],
 'kwargs': {'messages': [HumanMessage(content='Tell me a Short joke about Virat Kohli', additional_kwargs={}, response_metadata={})]}}

<h3>Model</h3>

In [13]:
message=llm.invoke(input=promptValue)
message

AIMessage(content='Why did Virat Kohli bring a ladder to the cricket match?\nBecause he heard the stumps were high and he wanted to be on top of his game!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 17, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CxSzzgRpNkZK3klCMSozTdJxCDPnj', 'finish_reason': 'stop', 'logprobs': None}, id='run--5dfc3c99-1e93-4da7-a91a-65456f57674b-0', usage_metadata={'input_tokens': 17, 'output_tokens': 33, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [15]:
message=llm.invoke(input=promptValue.to_messages())
message

AIMessage(content='Why did Virat Kohli bring a ladder to the cricket match?\n\nBecause he heard the match was going to be high-scoring!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 17, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CxT0hFrpurItP7lempU0oLvw9rUo1', 'finish_reason': 'stop', 'logprobs': None}, id='run--543eec4f-2220-4a08-900b-f35b5029a3bb-0', usage_metadata={'input_tokens': 17, 'output_tokens': 27, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [16]:
message=llm.invoke(input=promptValue.to_string())
message

AIMessage(content='Why did Virat Kohli bring a ladder to the cricket match? \n\nBecause he heard the bowlers were going to pitch it up!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 19, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CxT0rZXzYLY4AL1F1ABzZeTjsfR3i', 'finish_reason': 'stop', 'logprobs': None}, id='run--0b820fc6-80ec-4f6e-9c6a-110c3c197220-0', usage_metadata={'input_tokens': 19, 'output_tokens': 28, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

<h3> Output Parser </h3>

In [17]:
outParser.invoke(input=message)

'Why did Virat Kohli bring a ladder to the cricket match? \n\nBecause he heard the bowlers were going to pitch it up!'

<h3>RAG Example</h3>

In [19]:
embeddings=OpenAIEmbeddings() 

In [20]:
vectorStore=Chroma.from_texts(
    texts=["Harrison worked at kensho","bears like to eat honey"],
    embedding=embeddings
)

In [21]:
vectorStore.as_retriever()

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000020DD0C02B90>, search_kwargs={})

In [22]:
template="""
    Answer the question based only on the following context:
    {context}
    Question: {question}
"""

In [23]:
prompt=ChatPromptTemplate.from_template(
    template=template)

In [24]:
vectorStore.as_retriever().invoke(input="Who likes honey?")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='Harrison worked at kensho')]

In [25]:
RunnablePassthrough().invoke({"context":"How are you doing?"})

{'context': 'How are you doing?'}

In [27]:
setupAndRetrieval=RunnableParallel(
    {
        "context":vectorStore.as_retriever(),
        "question":RunnablePassthrough()
    }
)

In [28]:
chain=setupAndRetrieval|prompt|llm|outParser

In [30]:
question="Who worked at Kensho?"

In [31]:
k=time.time()
chain.invoke(input=question)
print(f"Time Taken: {time.time()-k} seconds")

Time Taken: 3.032109022140503 seconds


<h3>Without Using Runnable Parallel </h3>

In [32]:
chainTest=RunnablePassthrough.assign(context=itemgetter("question")|vectorStore.as_retriever())|prompt|llm|outParser

In [34]:
question="Who likes to eat honey??"

In [35]:
k=time.time()
chainTest.invoke(input={"question":question})
print(f"Time Taken: {time.time()-k} seconds")

Time Taken: 1.1296615600585938 seconds


<h3> Retriever Part </h3>

In [41]:
vectorStore.as_retriever().invoke(input=question)

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='Harrison worked at kensho')]

In [42]:
retrievedContent=setupAndRetrieval.invoke(input=question)
retrievedContent

{'context': [Document(metadata={}, page_content='bears like to eat honey'),
  Document(metadata={}, page_content='Harrison worked at kensho')],
 'question': 'Who likes to eat honey??'}

<h3>Prompt Part</h3>

In [43]:
promptValue=prompt.invoke(input=retrievedContent)
promptValue

ChatPromptValue(messages=[HumanMessage(content="\n    Answer the question based only on the following context:\n    [Document(metadata={}, page_content='bears like to eat honey'), Document(metadata={}, page_content='Harrison worked at kensho')]\n    Question: Who likes to eat honey??\n", additional_kwargs={}, response_metadata={})])

In [44]:
promptValue.to_messages()

[HumanMessage(content="\n    Answer the question based only on the following context:\n    [Document(metadata={}, page_content='bears like to eat honey'), Document(metadata={}, page_content='Harrison worked at kensho')]\n    Question: Who likes to eat honey??\n", additional_kwargs={}, response_metadata={})]

In [45]:
print(promptValue.to_string())

Human: 
    Answer the question based only on the following context:
    [Document(metadata={}, page_content='bears like to eat honey'), Document(metadata={}, page_content='Harrison worked at kensho')]
    Question: Who likes to eat honey??



<h3> LLM </h3>

In [ ]:
message=llm.invoke(input=promptValue)
message

AIMessage(content='Based on the given context, bears like to eat honey.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 58, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CxTRRGPPJ9pumRI26u8OmxrFgvsLE', 'finish_reason': 'stop', 'logprobs': None}, id='run--aeb7b7ae-eccf-4d55-b576-b78d321a4d1d-0', usage_metadata={'input_tokens': 58, 'output_tokens': 12, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [47]:
llm.invoke(input=promptValue.to_string())

AIMessage(content='Bears like to eat honey.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 60, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CxTRpSAvndZ6jTzKv8II80e1JNkGX', 'finish_reason': 'stop', 'logprobs': None}, id='run--54183ecf-ea08-49b4-bee4-eb6551d7ecd2-0', usage_metadata={'input_tokens': 60, 'output_tokens': 7, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [48]:
llm.invoke(input=promptValue.to_messages())

AIMessage(content='Based on the provided context, bears like to eat honey.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 58, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CxTSTRL9L2dCnULiVR0TCjjxleP4C', 'finish_reason': 'stop', 'logprobs': None}, id='run--ae1bffa0-e93a-401e-bde0-16d58affc856-0', usage_metadata={'input_tokens': 58, 'output_tokens': 12, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

<h3>Output Parser</h3>

In [49]:
output=outParser.invoke(input=message)
output

'Based on the given context, bears like to eat honey.'